In [5]:
import sys

from metatlas import metatlas_objects as metob
from metatlas import h5_query as h5q
from metatlas import gui as mgui

import qgrid
import metatlas_get_data_helper_fun as ma_data #look at it
import pandas as pd
import os
import tables
import pickle

%matplotlib notebook


In [6]:
mgui.show_experiments()

In [10]:
#Select the group
temp_group = metob.retrieve('Groups', name = '%_LS_%',username='*')
print len(temp_group)
# print temp_group
group = []
for i,g in enumerate(temp_group):
    if len(g.items) > 0:
        group.append(g)

mgui.edit_objects(group)

213


In [11]:
#Select the atlas
atlas = metob.retrieve('Atlas',name = '%_LS_%',username='*')
# mgui.edit_objects(atlas)
# len(atlas)
# atlas
from datetime import datetime, date

for i,a in enumerate(atlas):
    print i, a.name,  datetime.utcfromtimestamp(a.last_modified)

0 20151130_LS_Negative_Hilic_QExactive_Archetypes 2015-11-30 20:30:58
1 20151130_LS_Positive_Hilic_QExactive_Archetypes_ISTDs 2015-12-01 00:57:46
2 20151130_LS_Negative_Hilic_QExactive_Archetypes_ISTDs 2015-12-01 00:59:23
3 20151130_LS_Positive_Hilic_QExactive_Archetypes 2015-12-14 19:00:01


In [12]:
myAtlas = atlas[-1]

In [14]:
compound_list = []
for i in range(len(myAtlas.compound_identifications)):
    if myAtlas.compound_identifications[i].compound:
        compound_list.append(myAtlas.compound_identifications[i].compound[0].name)
    else:
        compound_list.append(myAtlas.compound_identifications[i].name)

cols = ['inchi',
 'mono_isotopic_molecular_weight',
 'creation_time',
 'description',
 'formula',
 'functional_sets',
 'last_modified',
 'reference_xrefs',
 'synonyms',
 'unique_id',
 'url',
 'username']
    
# print myAtlas[0].compound_identifications[0].compound
atlas_export = pd.DataFrame( index=compound_list, columns=cols)

atlas_export['name'] = compound_list
atlas_export.set_index('name',drop=True)
for i in range(len(myAtlas.compound_identifications)):
    if myAtlas.compound_identifications[i].compound:
        n = myAtlas.compound_identifications[i].compound[0].name
    else:
        n = myAtlas.compound_identifications[i].name
    if myAtlas.compound_identifications[i].compound:
        for c in cols:
                g = getattr(myAtlas.compound_identifications[i].compound[0],c)
                if g:
                    atlas_export.ix[n,c] = getattr(myAtlas.compound_identifications[i].compound[0],c)
    atlas_export.ix[n, 'label'] = myAtlas.compound_identifications[i].name
    atlas_export.ix[n,'rt_min'] = myAtlas.compound_identifications[i].rt_references[0].rt_min
    atlas_export.ix[n,'rt_max'] = myAtlas.compound_identifications[i].rt_references[0].rt_max
    atlas_export.ix[n,'rt_peak'] = myAtlas.compound_identifications[i].rt_references[0].rt_peak
    atlas_export.ix[n,'mz'] = myAtlas.compound_identifications[i].mz_references[0].mz
    atlas_export.ix[n,'mz_tolerance'] = myAtlas.compound_identifications[i].mz_references[0].mz_tolerance
    atlas_export.ix[n,'polarity'] = myAtlas.compound_identifications[i].mz_references[0].detected_polarity
atlas_export.to_csv('kate_atlas_uptake_pos.csv')


In [ ]:
# get and pickle everything This is MSMS, raw MS1 datapoints, compound, group info, and file info
# combine positive and negative mode atlas, by join atlases together vs two compound references
# typically file-groups are NOT split by polarity, files with un-matched polarity will be discarded for analysis of an identification
from metatlas import h5_query as h5q
import os
import tables
import dill
ma_data = reload(ma_data)
data = []
for i,treatment_groups in enumerate(group):
    for j in range(len(treatment_groups.items)):
        myFile = treatment_groups.items[j].hdf5_file
#         try:
#             rt_reference_index = int(treatment_groups.name[-1]) - 1
#         except:
#             rt_reference_index = 3
        print i,len(group),myFile
        row = []
        for compound in myAtlas.compound_identifications:
            result = {}
            result['lcmsrun'] = treatment_groups.items[j]
            result['group'] = treatment_groups
            result['identification'] = compound
            result['data'] = ma_data.get_data_for_a_compound(compound.mz_references[0],
                                    compound.rt_references[0],
                                    [ 'ms1_summary', 'eic', 'msms' ],
                                    myFile,0.2)
#                 print result['data']['ms1_summary']
            row.append(result)
        data.append(row)
with open('20160119_KZ_Positive_QE_HILIC_Avena_Uptake.pkl','w') as f:
    dill.dump(data,f)

0 27 /global/project/projectdirs/metatlas/raw_data/katezh/20160119_KZ_HILIC_Avena_exu_uptake/20160119_pHILIC___POS_MSMS_KZ_HA20_A_Nocardioides_sp_URHA0020______Run45.h5
0

In [16]:
%%javascript
var nb = IPython.notebook;
var kernel = IPython.notebook.kernel;
var command = "NOTEBOOK_FULL_PATH = '" + nb.base_url + nb.notebook_path + "'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [17]:
import os
filename = os.path.basename(NOTEBOOK_FULL_PATH)
%system cp $filename /project/projectdirs/openmsi/www/
temp = '%s/%s'%('/project/projectdirs/openmsi/www',filename)
%system chmod 775 $temp
print 'http://nbviewer.ipython.org/url/portal.nersc.gov/project/openmsi/%s?flush_cache=true'%filename

http://nbviewer.ipython.org/url/portal.nersc.gov/project/openmsi/Get a Group of Files and Atlas and Get Data.ipynb?flush_cache=true


In [ ]:
# helpful pandas hints for commands
# df
# print df.keys()
# df.loc[0,'name']
# df.filter(regex = 'name')
# df.iloc[0]
# for x in df.index:
#     print df.name[x]
#     if not metob.retrieve('Compounds',name=df.name[x]):
#         print df.name[x], "is not in database"
#     else:
#         print df.name[x], "Success"

#         
#     df = df.append(data)
#     print df.iloc[x,0]
#     print df.iloc[x]
#     print df.name[x]
# df.ix[(df['name']=='Adenine') == True]['mz']
# df.ix[(df['name']=='Adenine') == True]
# df.name.str.contains('Ad')
# df.name.str.contains('Ad').tolist()
# df

In [15]:
# files = metob.retrieve('lcmsrun',name='%Actino%')
import numpy as np
my_run = metob.retrieve('lcmsrun', hdf5_file='%20150510_C18_POS_MSMS_HE08-3%', username='*')
my_run
# for f in files:
#     print f.hdf5_file
with tables.open_file(my_run[0].hdf5_file) as fid:
    data = h5q.get_data(fid,2,1,min_rt = 2.1)#,max_rt = 17.9,min_precursor_MZ=633.12,max_precursor_MZ = 633.2)
print data['rt']
prt,pmz = get_unique_scan_data(data)
rt_cutoff = 0.23
mz_cutoff = 0.05
list_of_prt,list_of_pmz = get_non_redundant_precursor_list(prt,pmz,rt_cutoff,mz_cutoff)
#setup data format for searching
pactolus_input = {}
pactolus_input['spectra'] = []
pactolus_input['precursor_mz'] = []
for i,(prt,pmz) in enumerate(zip(list_of_prt,list_of_pmz)):
    idx = np.argwhere((data['precursor_MZ'] == pmz) & (data['rt'] == prt )).flatten()
    arr = np.array([data['mz'][idx], data['i'][idx]]).T
    pactolus_input['spectra'].append(arr)
    pactolus_input['precursor_mz'].append(pmz)
    


[  2.104074    2.104074    2.104074  ...,  11.5003891  11.5003891
  11.5003891]


In [20]:
import glob

# curr_ld_lib_path = ''
# os.environ['LD_LIBRARY_PATH'] = curr_ld_lib_path + ':/project/projectdirs/openmsi/jupyterhub_libs/boost_1_55_0/lib' + ':/project/projectdirs/openmsi/jupyterhub_libs/lib'
import sys
# sys.path.remove('/anaconda/lib/python2.7/site-packages')
sys.path.append('/global/project/projectdirs/openmsi/jupyterhub_libs/anaconda/lib/python2.7/site-packages')

# sys.path.insert(0,'/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages' )


sys.path.append('/project/projectdirs/openmsi/projects/meta-iq/pactolus/pactolus')

import score_frag_dag


pos_mode_neutralizations = [-1.00727646677, -(1.00727646677+1.00782504), +5.4857990946e-4,]
neg_mode_neutralizations = [-el for el in pos_mode_neutralizations]

# make lookup table
# path_to_trees = '/project/projectdirs/openmsi/projects/pactolus_trees/'
# all_my_h5_files = glob.glob('/project/projectdirs/openmsi/projects/pactolus_trees/*_hdf5_5_*.h5')

path_to_trees = '/project/projectdirs/openmsi/projects/ben_trees/'
all_my_h5_files = glob.glob('/project/projectdirs/openmsi/projects/ben_trees/*_hdf5_5_*.h5')

my_tree_filename = 'metacyc_max_depth_5'

if not os.path.isfile(os.path.join(path_to_trees, my_tree_filename + '.npy')):
    score_frag_dag.make_file_lookup_table_by_MS1_mass(all_my_h5_files, 
                                                      path=path_to_trees, 
                                                      save_result='metacyc_max_depth_5')

maxdepth_5_table = os.path.join(path_to_trees, my_tree_filename + '.npy')

params = {'file_lookup_table': maxdepth_5_table,
          'ms1_mass_tol': 0.05,
          'ms2_mass_tol': 0.05,
          'neutralizations': pos_mode_neutralizations,
          'max_depth': 5,
              }



In [21]:
print len(pactolus_input['spectra'])

580


In [39]:
import time

for i in range(10):
    start = time.time()
    foo = score_frag_dag.score_scan_list_against_trees([pactolus_input['spectra'][i]], [pactolus_input['precursor_mz'][i]], params)
    stop = time.time()
    print stop - start, np.sum(foo>0)
# np.argmax(foo)

0.00915908813477 0
0.225470066071 31
13.8581519127 24
0.199229955673 29
0.205362796783 13
0.293752908707 37
1.79723000526 133
0.200758218765 28
0.278684139252 48
0.00717997550964 0


In [27]:
metatlas_molecules = np.load(maxdepth_5_table)

In [33]:
metatlas_molecules.shape

(13190,)

In [25]:
foo.shape

(5, 13190)

In [ ]:
my_db = '/project/projectdirs/openmsi/projects/meta-iq/pactolus/data/' + 'MetaCyc.mdb'
pactolus_results = score_frag_dag.make_pactolus_hit_table(foo, maxdepth_5_table, original_db=my_db)

In [ ]:
for r in pactolus_results:
    if len(r)>0:
        print r[0]